<a href="https://colab.research.google.com/github/Melanee-Melanee/OCR/blob/main/easyocr_custum_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/JaidedAI/EasyOCR.git

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2736, done.
remote: Total 2736 (delta 0), reused 0 (delta 0), pack-reused 2736
Receiving objects: 100% (2736/2736), 157.83 MiB | 10.50 MiB/s, done.
Resolving deltas: 100% (1664/1664), done.
Updating files: 100% (313/313), done.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install easyocr
# easyocr version = 1.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [ ]:
%cd /content/EasyOCR/trainer/

/content/EasyOCR/trainer


In [ ]:
!pwd

/content


In [ ]:
!ls

all_data      craft	  model.py  __pycache__  saved_models  trainer.ipynb  utils.py
config_files  dataset.py  modules   README.md	 test.py       train.py


In [ ]:
!unzip /content/EasyOCR/trainer/all_data/en_train_filtered.zip

Archive:  /content/EasyOCR/trainer/all_data/en_train_filtered.zip
   creating: en_train_filtered/
  inflating: en_train_filtered/0.jpg  
  inflating: en_train_filtered/1.jpg  
  inflating: en_train_filtered/2.jpg  
  inflating: en_train_filtered/3.jpg  
  inflating: en_train_filtered/4.jpg  
  inflating: en_train_filtered/5.jpg  
  inflating: en_train_filtered/6.jpg  
  inflating: en_train_filtered/7.jpg  
  inflating: en_train_filtered/8.jpg  
  inflating: en_train_filtered/9.jpg  
  inflating: en_train_filtered/10.jpg  
  inflating: en_train_filtered/12.jpg  
  inflating: en_train_filtered/13.jpg  
  inflating: en_train_filtered/14.jpg  
  inflating: en_train_filtered/15.jpg  
  inflating: en_train_filtered/16.jpg  
  inflating: en_train_filtered/17.jpg  
  inflating: en_train_filtered/18.jpg  
  inflating: en_train_filtered/19.jpg  
  inflating: en_train_filtered/20.jpg  
  inflating: en_train_filtered/21.jpg  
  inflating: en_train_filtered/22.jpg  
  inflating: en_train_filtered/2

In [ ]:
!unzip /content/EasyOCR/trainer/all_data/en_val.zip

Archive:  /content/EasyOCR/trainer/all_data/en_val.zip
   creating: en_val/
  inflating: en_val/labels.csv       
  inflating: en_val/0.jpg            
  inflating: en_val/1.jpg            
  inflating: en_val/2.jpg            
  inflating: en_val/3.jpg            
  inflating: en_val/4.jpg            
  inflating: en_val/5.jpg            
  inflating: en_val/6.jpg            
  inflating: en_val/7.jpg            
  inflating: en_val/8.jpg            
  inflating: en_val/9.jpg            
  inflating: en_val/10.jpg           
  inflating: en_val/12.jpg           
  inflating: en_val/13.jpg           
  inflating: en_val/14.jpg           
  inflating: en_val/15.jpg           
  inflating: en_val/16.jpg           
  inflating: en_val/17.jpg           
  inflating: en_val/18.jpg           
  inflating: en_val/19.jpg           
  inflating: en_val/20.jpg           
  inflating: en_val/21.jpg           
  inflating: en_val/22.jpg           
  inflating: en_val/23.jpg           
  inflating:

In [ ]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [ ]:
cudnn.benchmark = True
cudnn.deterministic = False

In [ ]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [ ]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['en_train_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: en_train_filtered
all_data/en_train_filtered
sub-directory:	/en_train_filtered	 num samples: 882
num total samples of en_train_filtered: 882 x 1.0 (total_data_usage_ratio) = 882
num samples of en_train_filtered per batch: 3 x 1.0 (batch_ratio) = 3
--------------------------------------------------------------------------------
Total_batch_size: 3 = 3
--------------------------------------------------------------------------------
dataset_root:    all_data/en_val	 dataset: /
all_data/en_val/
sub-directory:	/.	 num samples: 882
------------------------------------------

/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


training time:  26.74246311187744
[20/30] Train loss: 15.94893, Valid loss: 4.64690, Elapsed_time: 26.74388
Current_accuracy : 0.000, Current_norm_ED  : 0.0000
Best_accuracy    : 0.000, Best_norm_ED     : 0.0000
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
Geometric ! functioned Camino |                           | 0.0000	False
Kilbride + HERMITAGE +    |                           | 0.0108	False
--------------------------------------------------------------------------------
validation time:  117.45002579689026
end the training


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!cp /root/EasyOCR//user_network/custom_example.yaml /root/.EasyOCR//user_network/custom_example.yaml

In [ ]:
!cp /root/EasyOCR/user_network/custom_example.py /root/.EasyOCR/user_network/custom_example.py

In [ ]:
!cp /root/EasyOCR/model/custom_example.pth /root/.EasyOCR/model/custom_example.pth

In [ ]:
#single line:
import easyocr
reader = easyocr.Reader(['en'], recog_network='custom_example')
result = reader.readtext('/content/999.jpg', detail=0)
print(result)

In [ ]:
#multiple lines:
import easyocr
reader = easyocr.Reader(['en'],recog_network='custom_example' )

In [ ]:
#checking normal images
text = ''
results = reader.readtext('/content/text1.png')
for result in results:
  text = text + result[1] +  ' '

text = text[:-1]
print(text)

This is the first line of tthis text example. This is the second line of the same text.


In [ ]:
#checking normal images
text = ''
results = reader.readtext('/content/text2.png')
for result in results:
  text = text + result[1] +  ' '

text = text[:-1]
print(text)

It was the best of times;, it was the worst of times, it was the age of wisdom, it was the age of foolishness..


In [3]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3